# RAG-Pipeline

In this notebook, we will demonstrate how to use the RAG pipeline to generate answers for the questions defined in the cleantech evaluation dataset. The pipeline consists of the following steps:
1. Data Preparation and Ingestion
2. Embedding
3. Store Embeddings in Vector Database
4. Retrieve Contexts
5. Generate Prompts

## Imports

In [11]:
import pandas as pd
import time
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from openai import AzureOpenAI
import openai

import credentials

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import math

In [2]:
deployment_name='gpt-4' #This will correspond to the custom name you chose for your deployment when you deployed a model. 

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2023-12-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

## Data Preparation and Ingestion

We will load and prepare the data, create embeddings for each chunk, and store the embeddings in a vector database. Since the data is already cleaned, we'll move directly to chunking and embedding.

In [3]:
data = pd.read_csv('../data/processed/cleantech_processed.csv')
data_eval = pd.read_csv('../data/evaluation/cleantech_rag_evaluation_data_2024-09-20.csv', delimiter=";")

data.head()

,title,date,content,domain,url
0,Qatar to Slash Emissions as LNG Expansion Adva...,2021-01-13,Qatar Petroleum (QP) is targeting aggressive c...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...
1,India Launches Its First 700 MW PHWR,2021-01-15,Nuclear Power Corp of India Ltd (NPCIL) synchr...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...
2,New Chapter for US-China Energy Trade,2021-01-20,New US President Joe Biden took office this we...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...
3,Japan: Slow Restarts Cast Doubt on 2030 Energy...,2021-01-22,The slow pace of Japanese reactor restarts con...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...
4,NYC Pension Funds to Divest Fossil Fuel Shares,2021-01-25,Two of New York City's largest pension funds s...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...


In [4]:
print("Number of articles in the dataset: ", len(data))

Number of articles in the dataset:  9593


- `chunk_size`: Sets the maximum character length of each chunk to 1000 characters.
- `chunk_overlap`: Adds a n-character overlap between chunks to preserve context.

In [ ]:
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
EMBEDDING_MODEL = "text-embedding-ada-002"

In [5]:
def chunk_text(dataframe, text_column, chunk_size=1000, chunk_overlap=100):
    # Initialize RecursiveCharacterTextSplitter with dynamic parameters
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    # Split text in the specified column
    dataframe['content_chunks'] = dataframe[text_column].apply(lambda text: text_splitter.split_text(text))
    
    # Flatten the DataFrame for individual chunk rows
    chunked_df = dataframe.explode('content_chunks').reset_index(drop=True)

    return chunked_df

# Apply the chunking with adjustable parameters
chunked_data = chunk_text(data, text_column='content', chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
chunked_data.head()

,title,date,content,domain,url,content_chunks
0,Qatar to Slash Emissions as LNG Expansion Adva...,2021-01-13,Qatar Petroleum (QP) is targeting aggressive c...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,Qatar Petroleum (QP) is targeting aggressive c...
1,Qatar to Slash Emissions as LNG Expansion Adva...,2021-01-13,Qatar Petroleum (QP) is targeting aggressive c...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,eyeing a phased expansion to 126 million tons/...
2,Qatar to Slash Emissions as LNG Expansion Adva...,2021-01-13,Qatar Petroleum (QP) is targeting aggressive c...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"estimates to be worth around $ 35 billion, is ..."
3,India Launches Its First 700 MW PHWR,2021-01-15,Nuclear Power Corp of India Ltd (NPCIL) synchr...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,Nuclear Power Corp of India Ltd (NPCIL) synchr...
4,India Launches Its First 700 MW PHWR,2021-01-15,Nuclear Power Corp of India Ltd (NPCIL) synchr...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"start up, although order flows will depend on ..."


In [6]:
len(chunked_data)

44795

Now that we have chunked the texts into smaller segments, the next step is to pass these chunks through an embedding model to obtain their vector representations. The embedding model maps the textual information into high-dimensional vector spaces, where semantic similarities and relationships are preserved.

## Embedding Creation

In [7]:
BATCH_SIZE = 50
MAX_WORKERS = 8  # Number of threads
RETRY_DELAY = 60  # Retry delay in seconds for rate-limiting errors

# Function to generate embeddings for a batch of texts with retry logic
def embed_batch(text_batch, model, retries=3):
    attempts = 0
    while attempts < retries:
        try:
            response = client.embeddings.create(input=text_batch, model=model)
            return [item.embedding for item in response.data]
        except Exception as e:
            if "rate limit" in str(e).lower() or "429" in str(e):
                print(f"Rate limit exceeded. Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)  # Wait before retrying
                attempts += 1
            else:
                print(f"Error embedding batch: {e}")
                return [None] * len(text_batch)
    print("Failed to embed after multiple attempts.")
    return [None] * len(text_batch)

# Function to process batches in parallel with rate limiting
def embed_in_batches(data, model=EMBEDDING_MODEL, batch_size=BATCH_SIZE, max_workers=MAX_WORKERS):
    total_chunks = len(data)
    num_batches = math.ceil(total_chunks / batch_size)
    
    # Create batches
    batches = [data[i * batch_size: (i + 1) * batch_size] for i in range(num_batches)]
    embeddings = [None] * total_chunks  # Placeholder for embeddings

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {}
        for i, batch in enumerate(batches):
            futures[executor.submit(embed_batch, batch, model)] = i
        
        # Process results as they complete
        for future in tqdm(as_completed(futures), total=len(futures), desc="Embedding in Parallel"):
            batch_index = futures[future]
            try:
                batch_embeddings = future.result()
                embeddings[batch_index * batch_size: (batch_index + 1) * batch_size] = batch_embeddings
            except Exception as e:
                print(f"Error processing batch {batch_index}: {e}")
    
    return embeddings

# Prepare data for embedding generation
texts_to_embed = chunked_data["content_chunks"].tolist()

# Run batch embedding generation with rate limiting
chunked_data['embeddings'] = embed_in_batches(
    texts_to_embed, 
    embedding_model=EMBEDDING_MODEL,
    batch_size=BATCH_SIZE,
    max_workers=MAX_WORKERS
)

Generating Embeddings in Parallel:  57%|█████▋    | 509/896 [1:07:10<35:21,  5.48s/it]  

Error generating embeddings for batch: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-12-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 60 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}
Error generating embeddings for batch: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-12-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 60 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}


Generating Embeddings in Parallel:  64%|██████▍   | 573/896 [1:15:14<58:03, 10.78s/it]  

Error generating embeddings for batch: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-12-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}
Error generating embeddings for batch: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-12-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}


Generating Embeddings in Parallel:  97%|█████████▋| 866/896 [1:52:44<02:22,  4.74s/it]  

Error generating embeddings for batch: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-12-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 60 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}


Generating Embeddings in Parallel: 100%|██████████| 896/896 [1:56:46<00:00,  7.82s/it]


In [8]:
print("Dimension of the embeddings: ", len(chunked_data['embeddings'][0]))

Dimension of the embeddings:  1536


The `chunked_data` DataFrame is structured so that each row represents an individual **chunk** of text derived from the original documents, with a corresponding **embedding vector** stored in the `embeddings` column. Each embedding vector is a dense list of floating-point numbers (1536 dimensions in our case, given the use of the `text-embedding-ada-002` model) that encapsulates the semantic meaning of the text chunk.

These embeddings are critical for the RAG system, as they allow efficient similarity searches and retrieval tasks by representing the content of each text chunk in a high-dimensional vector space. When a query is embedded, the RAG system can quickly locate the most relevant text chunks by identifying embedding vectors in the database that are closest in meaning. This approach enables the system to retrieve contextually relevant information by comparing semantic relationships, streamlining the entire retrieval process.

In [ ]:
import pickle

# Save embeddings and corresponding chunked data to a file
with open('../recursive_{CHUNK_SIZE}_chunksize_{CHUNK_OVERLAP}_overlap_{EMBEDDING_MODEL}.pkl', 'wb') as f:
    pickle.dump(chunked_data, f)

print("Embeddings and chunked data saved")